In [1]:
import org.apache.spark.sql.SparkSession;

//import statistics.functions._;

import org.apache.spark.sql.DataFrame;
import org.apache.spark.sql.types._;
import org.apache.spark.sql.functions._;
import org.apache.spark.ml.Pipeline;
import org.apache.spark.ml.Model;
import org.apache.spark.ml.classification.{LogisticRegression, LinearSVC, OneVsRest};
import org.apache.spark.ml.evaluation.{BinaryClassificationEvaluator, MulticlassClassificationEvaluator};
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.ml.feature.Binarizer;
import org.apache.spark.ml.feature.{RegexTokenizer, NGram};
import org.apache.spark.ml.feature.{HashingTF, IDF};
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder};

Intitializing Scala interpreter ...

Spark Web UI available at http://pranav-pc:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1581250932020)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.Model
import org.apache.spark.ml.classification.{LogisticRegression, LinearSVC, OneVsRest}
import org.apache.spark.ml.evaluation.{BinaryClassificationEvaluator, MulticlassClassificationEvaluator}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.ml.feature.Binarizer
import org.apache.spark.ml.feature.{RegexTokenizer, NGram}
import org.apache.spark.ml.feature.{HashingTF, IDF}
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}


In [2]:
val path: String = "hdfs://localhost:9000/TextMining/tokens/";

// "rate" must be Double because it can be easily binarized by Spark
val original_schema = new StructType(Array(
  StructField("product",          StringType,  true),
  StructField("votes",            IntegerType, true),
  StructField("rate",             DoubleType,  true),
  StructField("userID",    StringType,  true),
  StructField("original_text",    StringType,  true),
  StructField("text",             StringType,  true),
  StructField("summary",          StringType,  true)));

val original_data:DataFrame = spark.read
  .options(Map("delimiter" -> "\t"))
  .schema(original_schema)
  .csv(path)
  .na.drop();

path: String = hdfs://localhost:9000/TextMining/tokens/
original_schema: org.apache.spark.sql.types.StructType = StructType(StructField(product,StringType,true), StructField(votes,IntegerType,true), StructField(rate,DoubleType,true), StructField(userID,StringType,true), StructField(original_text,StringType,true), StructField(text,StringType,true), StructField(summary,StringType,true))
original_data: org.apache.spark.sql.DataFrame = [product: string, votes: int ... 5 more fields]


In [3]:
original_data.show(5)

+----------+-----+----+--------------+--------------------+--------------------+--------------------+
|   product|votes|rate|        userID|       original_text|                text|             summary|
+----------+-----+----+--------------+--------------------+--------------------+--------------------+
|0143065971|    1| 5.0|A3JXKC2CMCA9MR|Great condition a...| great condit great |          Five Stars|
|0143065971|    4| 5.0| A18GG84RF7J2B|This is a masterp...|masterpiec someon...|Fantastic Book Ab...|
|1423600150|    1| 5.0|A2VCOE59QZ9967|Great little cook...|great littl cookb...|           Delicious|
|1423600150|    1| 5.0|A1DBZ3TCVMCQW2|   Great mexi stuff.|   great mexi stuff |          Five Stars|
|1423600150|    1| 5.0| AYXIWMJSXU3PW|          Great book|              great |          Five Stars|
+----------+-----+----+--------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [4]:
/* original_data.select("summary","rate","votes")
 *    .groupBy("summary")
 *    .agg(mean("rate"), mean("votes"), count("summary"))
 *   .orderBy(desc("count(summary)"))
 */

# Logistic Regression

In [5]:
val binarizer = new Binarizer()
  .setInputCol("rate")
  .setOutputCol("label")
  .setThreshold(3.5);

// get n-grams
val tokenizer = new RegexTokenizer()
  .setInputCol("text")
  .setOutputCol("tokens")
  .setPattern("\\W");
val ngrams = new NGram()
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("n-grams");

// calc tf-idf 
val tf = new HashingTF()
  .setInputCol(ngrams.getOutputCol)
  .setOutputCol("tf");
val idf = new IDF()
  .setInputCol(tf.getOutputCol)
  .setOutputCol("tf-idf")
  .setMinDocFreq(3);

// build the classifier
val classifierMod = new LogisticRegression()
  .setMaxIter(10)
  .setFeaturesCol(idf.getOutputCol)
  .setLabelCol(binarizer.getOutputCol);

// this is the pipeline that data follows to be evaluated
val pipeline = new Pipeline()
  .setStages(Array(binarizer, tokenizer, ngrams, tf, idf, classifierMod));

// a little of optimization: try different hyperparameters
val paramGrid = new ParamGridBuilder()
  .addGrid(classifierMod.regParam, Array(0.01, 0.05, 0.1))
  .addGrid(ngrams.n, Array(1, 2, 3))
  .build();

// do it with a cross validation on the train set (3 folds)
val evaluator = new MulticlassClassificationEvaluator() //è f1 sulla classe 0.0

val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3);

binarizer: org.apache.spark.ml.feature.Binarizer = Binarizer: uid=binarizer_3e35340c4b7f
tokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_65d53eba131b
ngrams: org.apache.spark.ml.feature.NGram = NGram: uid=ngram_23d490d70b8f, n=2
tf: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_cca4b7701211, binary=false, numFeatures=262144
idf: org.apache.spark.ml.feature.IDF = idf_be39df358ffb
classifierMod: org.apache.spark.ml.classification.LogisticRegression = logreg_ee64752c37b0
pipeline: org.apache.spark.ml.Pipeline = pipeline_f2832bcc437b
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	ngram_23d490d70b8f-n: 1,
	logreg_ee64752c37b0-regParam: 0.01
}, {
	ngram_23d490d70b8f-n: 2,
	logreg_ee64752c37b0-regParam: 0.01
}, {
	ngram_23d490d70b8f-n: 3,
	l...


In [6]:
// spark.sql("SELECT summary, COUNT(*) FROM original_data")

In [7]:
println("Training... ");
val model = cv.fit(original_data);
println("done!");

// print results
for (i <- 0 until model.avgMetrics.size) {
  println("\n\n");
  println(model.getEstimatorParamMaps(i));
  println(model.avgMetrics(i));
}

Training... 
done!



{
	ngram_23d490d70b8f-n: 1,
	logreg_ee64752c37b0-regParam: 0.01
}
0.8767617577593136



{
	ngram_23d490d70b8f-n: 2,
	logreg_ee64752c37b0-regParam: 0.01
}
0.8845783557809365



{
	ngram_23d490d70b8f-n: 3,
	logreg_ee64752c37b0-regParam: 0.01
}
0.813688775592737



{
	ngram_23d490d70b8f-n: 1,
	logreg_ee64752c37b0-regParam: 0.05
}
0.8564910958427848



{
	ngram_23d490d70b8f-n: 2,
	logreg_ee64752c37b0-regParam: 0.05
}
0.8727693110974878



{
	ngram_23d490d70b8f-n: 3,
	logreg_ee64752c37b0-regParam: 0.05
}
0.8037707567982274



{
	ngram_23d490d70b8f-n: 1,
	logreg_ee64752c37b0-regParam: 0.1
}
0.8405928493000251



{
	ngram_23d490d70b8f-n: 2,
	logreg_ee64752c37b0-regParam: 0.1
}
0.860296788185761



{
	ngram_23d490d70b8f-n: 3,
	logreg_ee64752c37b0-regParam: 0.1
}
0.7933461211257044


model: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_3756532dad18, bestModel=pipeline_f2832bcc437b, numFolds=3


# SVM multiclassifier

In [8]:
// get n-grams
val tokenizer = new RegexTokenizer()
  .setInputCol("text")
  .setOutputCol("tokens")
  .setPattern("\\W");
val ngrams = new NGram()
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("n-grams");

// calc tf-idf 
val tf = new HashingTF()
  .setInputCol(ngrams.getOutputCol)
  .setOutputCol("tf");
val idf = new IDF()
  .setInputCol(tf.getOutputCol)
  .setOutputCol("tf-idf")
  .setMinDocFreq(3);

// build the classifier
val svm = new LinearSVC();

val classifierMod = new OneVsRest()
  .setClassifier(svm)
  .setFeaturesCol(idf.getOutputCol)
  .setLabelCol("rate");

// this is the pipeline that data follows to be evaluated
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, ngrams, tf, idf, classifierMod));

// a little of optimization: try different hyperparameters
val paramGrid = new ParamGridBuilder()
  .addGrid(svm.regParam, Array(0.1, 0.5))
  .addGrid(ngrams.n, Array(1, 2, 3))
  .build();

val evaluator = new MulticlassClassificationEvaluator()
                        .setMetricName("accuracy")
                        .setLabelCol("rate")

// do it with a cross validation on the train set (3 folds)
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3);

tokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_fac8c4b89e51
ngrams: org.apache.spark.ml.feature.NGram = NGram: uid=ngram_84dd183c53be, n=2
tf: org.apache.spark.ml.feature.HashingTF = HashingTF: uid=hashingTF_98d8d6790dfb, binary=false, numFeatures=262144
idf: org.apache.spark.ml.feature.IDF = idf_b4687644f064
svm: org.apache.spark.ml.classification.LinearSVC = linearsvc_dfa2ea2e229c
classifierMod: org.apache.spark.ml.classification.OneVsRest = oneVsRest_5ea4752e8604
pipeline: org.apache.spark.ml.Pipeline = pipeline_2493a742fb86
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	ngram_84dd183c53be-n: 1,
	linearsvc_dfa2ea2e229c-regParam: 0.1
}, {
	ngram_84dd183c53be-n: 1,
	linearsvc_dfa2ea2e229c-regParam: 0.5
}, {
	ngram_84dd183c53be-n: 2,
	linearsvc_dfa2ea2...


In [9]:
print("Training... ");
val model = cv.fit(original_data)
print("done!")

Training... done!

model: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_9bd91ae77766, bestModel=pipeline_2493a742fb86, numFolds=3


In [10]:
for (i <- 0 until model.avgMetrics.size) {
  println("\n\n");
  println(model.getEstimatorParamMaps(i));
  println(model.avgMetrics(i));
}




{
	ngram_84dd183c53be-n: 1,
	linearsvc_dfa2ea2e229c-regParam: 0.1
}
0.6444849701057255



{
	ngram_84dd183c53be-n: 1,
	linearsvc_dfa2ea2e229c-regParam: 0.5
}
0.6252414319347207



{
	ngram_84dd183c53be-n: 2,
	linearsvc_dfa2ea2e229c-regParam: 0.1
}
0.6603150472436443



{
	ngram_84dd183c53be-n: 2,
	linearsvc_dfa2ea2e229c-regParam: 0.5
}
0.6402047012108124



{
	ngram_84dd183c53be-n: 3,
	linearsvc_dfa2ea2e229c-regParam: 0.1
}
0.6199566142785332



{
	ngram_84dd183c53be-n: 3,
	linearsvc_dfa2ea2e229c-regParam: 0.5
}
0.6139563253890734


In [12]:
original_data.groupBy("rate").count().show()

+----+-------+
|rate|  count|
+----+-------+
| 1.0| 258294|
| 4.0|1197765|
| 3.0| 475678|
| 2.0| 213119|
| 5.0|3319659|
+----+-------+



In [14]:
val results = model.bestModel.transform(original_data)

results: org.apache.spark.sql.DataFrame = [product: string, votes: int ... 10 more fields]


In [21]:
val predictionsAndLabels = results.select("prediction", "rate")
  .map(row => (row.getDouble(0), row.getDouble(1)))

val metrics = new MulticlassMetrics(predictionsAndLabels.rdd)

val confusionMatrix = metrics.confusionMatrix

predictionsAndLabels: org.apache.spark.sql.Dataset[(Double, Double)] = [_1: double, _2: double]
metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@5d7d9091
confusionMatrix: org.apache.spark.mllib.linalg.Matrix =
125244.0  3577.0   7820.0    2816.0    118816.0
31162.0   47076.0  24203.0   8216.0    102431.0
14199.0   6061.0   134585.0  49938.0   270750.0
4772.0    2617.0   26373.0   207166.0  956442.0
4118.0    1505.0   8109.0    42067.0   3263300.0


In [54]:
println(s"F1 measure  ${metrics.weightedFMeasure}")

println(s"weightedRecall  ${metrics.weightedRecall}")

println(s"weightedPrecision  ${metrics.weightedPrecision}")

F1 measure  0.6289661678651075
weightedRecall  0.6912545761151722
weightedPrecision  0.6885475253460855


In [22]:
confusionMatrix

res10: org.apache.spark.mllib.linalg.Matrix =
125244.0  3577.0   7820.0    2816.0    118816.0
31162.0   47076.0  24203.0   8216.0    102431.0
14199.0   6061.0   134585.0  49938.0   270750.0
4772.0    2617.0   26373.0   207166.0  956442.0
4118.0    1505.0   8109.0    42067.0   3263300.0


In [4]:
%%python

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

confusionMatrix = np.array([[125244.0, 3577.0,  7820.0,   2816.0,   118816.0],
                            [31162.0,  47076.0, 24203.0,  8216.0,   102431.0],
                            [14199.0,  6061.0,  134585.0, 49938.0,  270750.0],
                            [4772.0,   2617.0,  26373.0,  207166.0, 956442.0],
                            [4118.0,   1505.0,  8109.0,   42067.0,  3263300.0]])

In [12]:
%%python
#sort of precision
sns.heatmap(confusionMatrix/np.sum(confusionMatrix, axis=0), annot=True, cmap="Blues",
           xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

In [21]:
%%python
#sort of recall
sns.heatmap((confusionMatrix.swapaxes(0,1)/np.sum(confusionMatrix, axis=1)).swapaxes(0,1), 
            annot=True, cmap="Blues",
           xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

In [20]:
%%python

sns.heatmap(confusionMatrix, annot=True, cmap="Blues",
           xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()